In [1]:
import re
import time
import numpy as np
import pandas as pd
from appium import webdriver
from appium.options.android import UiAutomator2Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [5]:
class Reel_View:
    def __init__(self, version, adb_name, file_name, username):
        self.version = version
        self.adb_name = adb_name
        self.file_name = file_name
        self.username = username
        self.driver = self.initialize_driver()
        self.urls = self.file_read()
        self.views = None
    

    def initialize_driver(self):
        desired_caps = {
            'platformName': 'Android',
            'deviceName': str(self.adb_name),  # Replace with your device name
            'udid': str(self.adb_name),  # Replace with your device UDID
            'platformVersion': str(self.version),  # Replace with your Android version (e.g., 11)
            'appPackage': 'com.instagram.android',
            'appActivity': '.activity.MainTabActivity',
            'automationName': 'UiAutomator2',  # Use UiAutomator2 for Android
            'noReset': True,  # Keeps the app data between sessions
            'newCommandTimeout': 6000,  # Timeout for new commands to the server
            'adbExecTimeout': 20000,  # Timeout for ADB commands (adjust as needed)
            'autoGrantPermissions': True,  # Grant necessary permissions automatically
            'disableWindowAnimation': True,  # Disable window animations for faster test execution
            'unicodeKeyboard': True,  # Enable Unicode input (if needed)
            'resetKeyboard': True,  # Reset keyboard after test (if needed)
            "appium:ensureWebviewsHavePages": True
        }
        options = UiAutomator2Options().load_capabilities(caps=desired_caps)
        return webdriver.Remote('http://127.0.0.1:4723/wd/hub', options=options)

    def file_read(self):
        file = pd.read_excel(str(self.file_name))
        urls = file['Urls']
        return urls
    
    def view_collector(self):

        self.views = []
        messages = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, '//android.widget.Button[@content-desc="No unread messages"]')))
        messages.click()

        chat_account = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//android.widget.TextView[@content-desc="{self.username}"]')))
        chat_account.click()
        for url in self.urls:
            chat_edit = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//android.widget.EditText[@resource-id="com.instagram.android:id/row_thread_composer_edittext"]')))
            chat_edit.click()
            chat_edit.send_keys(url)

            send = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//android.widget.ImageView[@content-desc="Send"]')))
            send.click()

            try:
                reel = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, f'(//android.widget.FrameLayout[@resource-id="com.instagram.android:id/context_reply_context_message_container"])[2]')))
                reel.click()
            except:
                continue
            likes = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//android.view.ViewGroup[contains(@content-desc, 'Like number is') and contains(@content-desc, 'View likes')]")))
            likes.click()

            view = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.ID, f'com.instagram.android:id/video_view_count_text')))
            self.views.append(view.text)
            self.driver.back()
            self.driver.back()
            

    def dataframe(self):
        self.view_collector()
        data = pd.DataFrame({
            'URLs': self.urls,
            'Views': self.views
        })
        data.to_excel('output.xlsx', index=False)
        self.driver.quit()


In [6]:
test = Reel_View("14", "R9ZWA07928P", "Reel URLs.xlsx", "Prince Pride")
test.dataframe()